# Implementation of a Newton–Raphson solver for nonlinear elasticity

In this notebook, we will analyse the same problem as in the previous notebook [Hyperelastic.ipynb](Hyperelastic.ipynb). However, this time, we will implement or own Newton–Raphson solver, rather than rely on FEniCS's `NonlinearVariationalSolver`.

## Setting up the problem

The code below is merely a copy/paste of the code from the previous sessions, as we use the same problem to illustrate the implementation of the Newton–Raphson iterations

We first import and setup the usual modules.

In [ ]:
import dolfin
import ufl
import numpy as np
import matplotlib.pyplot as plt
import os.path

%matplotlib inline

dolfin.parameters["form_compiler"]["cpp_optimize"] = True
dolfin.parameters["form_compiler"]["representation"] = "uflacs"
plt.style.use("seaborn-notebook")

### Parameters of the simulation

Place here the parameters that can be changed without altering the logics of the code.

In [ ]:
Lx, Ly = 1.0, 0.1 # Dimensions of the beam
nx, ny = 20, 5 # Number of elements in each direction
Y, nu = 1e3, 0.3 # Young modulus and Poisson ratio
load_min, load_max, nsteps = 0.0, 0.3, 40 # Loading schedule
degree = 2

output_dir = "nr_output"

You should not alter the cell below

In [ ]:
loads = np.linspace(load_min, load_max,nsteps)

mu = dolfin.Constant(Y/(2*(1 + nu)))
lmbda = dolfin.Constant(Y*nu/((1 + nu)*(1 - 2*nu)))
lmbda = 2*lmbda*mu/(lmbda + 2*mu) 

### Mesh

In [ ]:
mesh = dolfin.RectangleMesh(dolfin.Point(0,0),
                            dolfin.Point(Lx,Ly),
                            nx, ny)
left = dolfin.CompiledSubDomain("near(x[0],0) && on_boundary")
right = dolfin.CompiledSubDomain("near(x[0],Lx) && on_boundary", Lx=Lx)

boundary_markers = dolfin.MeshFunction("size_t", mesh, 1, 0)
boundary_indices = {"left": 1, "right": 2}
left.mark(boundary_markers, boundary_indices["left"])
right.mark(boundary_markers, boundary_indices["right"])

ds = dolfin.ds(domain=mesh,subdomain_data=boundary_markers)
dx = dolfin.dx(domain=mesh)

### Function space

In [ ]:
V_element = dolfin.VectorElement("CG", mesh.ufl_cell(), degree=degree) 
V = dolfin.FunctionSpace(mesh, V_element)
u = dolfin.Function(V, name="u")

### Potential energy

We first define the strain energy of a (compressible) neo-Hookean material.

In [ ]:
I = ufl.Identity(2)    
F = ufl.variable(I+ufl.grad(u))  
C = F.T*F                   
Ic = ufl.tr(C)
J  = ufl.det(F)
E = 1/2*(C-I)

psi = (mu/2)*(Ic-2)-mu*ufl.ln(J)+(lmbda/2)*(ufl.ln(J))**2

We then define the loading (body forces $\mathbf{B}$).

In [ ]:
B = dolfin.Expression(("0.0", "mu*t"), t=0, mu=mu, degree=0)
T = mu*dolfin.Constant((0.0, 0.0))

We can now define the potential energy and its derivatives w.r.t the displacement $\mathbf{u}$.

In [ ]:
potential_energy = psi*dx-ufl.dot(B, u)*dx-ufl.dot(T, u)*ds(boundary_indices['right'])
residual = ufl.derivative(potential_energy, u, dolfin.TestFunction(V))
jacobian = ufl.derivative(residual, u, dolfin.TrialFunction(V))

Boundary conditions for a built-in support on the left-hand side.

In [ ]:
bcs = [dolfin.DirichletBC(V, (0.,0.), left)]

### Computing the reference solution

We use the built-in non-linear solver to compute a reference solution, which we will compare to the solution computed with our own NR solver.

In [ ]:
problem = dolfin.NonlinearVariationalProblem(residual, u, bcs=bcs, J=jacobian)
solver = dolfin.NonlinearVariationalSolver(problem)
displ_ref = np.zeros_like(loads)
u.interpolate(dolfin.Constant((0., 0.)))
file_u = dolfin.XDMFFile(os.path.join(output_dir, "u_ref.xdmf"))
file_u.parameters.update({"flush_output":True,
                          "functions_share_mesh":True,
                          "rewrite_function_mesh":False})
for (i, t) in enumerate(loads):
    B.t = -t
    solver.solve()
    displ_ref[i] = dolfin.assemble(u[1]*ds(boundary_indices["right"]))/Ly 
    #print("t={:+3.3f}, u={:+3.3f}".format(t,displ_ref[i]))
    with file_u as file:
        file.write(u,t)
plt.plot(loads,displ_ref, "o")
plt.xlabel("load")
plt.ylabel("end-displacement")

## Implementation of the Newton–Raphson solver

We will discuss the statement below in class.

In [ ]:
bcs_h = bcs

In [ ]:
def simple_monitor(iteration, u, norm_u, norm_delta_u, norm_residual):
    print("Iteration: {:3d}, Error: {:3.4e}, Residual: {:3.4e}".format(iteration, norm_delta_u, norm_residual))
        
def plot_monitor(iteration, u, norm_u, norm_delta_u, norm_residual):
    simple_monitor(iteration, u, norm_u, norm_delta_u, norm_residual)
    plt.figure()
    dolfin.plot(u, mode="displacement") 

def newton_solver(u, max_iter=100, rtol=1e-6, atol=1e-6, monitor=None):
    delta_u = dolfin.Function(V)
    delta_u.interpolate(dolfin.Constant((0., 0.)))
    for k in range(max_iter):
        # Solve the linearized problem for the increment delta_u with homogenous BCs
        linear_problem = dolfin.LinearVariationalProblem(jacobian, -residual, delta_u, bcs_h)
        linear_solver = dolfin.LinearVariationalSolver(linear_problem)
        linear_solver.solve()
        # Update the solution
        u.assign(u+delta_u)
        # Stopping criterion based on the L2 norm of u and delta_u
        norm_delta_u = dolfin.norm(delta_u, "L2")
        norm_u = dolfin.norm(u, "L2")
        R_vec = dolfin.assemble(residual)
        for bc in bcs_h:
            bc.apply(R_vec)
        norm_residual = dolfin.norm(R_vec, "L2")
        if monitor is not None:
            monitor(k, u, norm_u, norm_delta_u, norm_residual)
        if norm_delta_u <= rtol*norm_u+atol:
            break
    else:
        # See for-else statement:
        # https://book.pythontips.com/en/latest/for_-_else.html#else-clause
        # This block is entered only if the above loop completes,
        # which means that the maximum number of iterations has been reached.
        raise RuntimeError("could not converge, norm_u {}, norm_delta_u {}".format(norm_u, norm_delta_u)) 
    return k

We are now ready to call our solver, first with a very small load, so that the solution is nearly that of the linear problem, and the algorithm should converge in one iteration.

In [ ]:
u.interpolate(dolfin.Constant((0., 0.)))
B.t = load_max/1e4
newton_solver(u, monitor=simple_monitor)

Let's try a slightly higher load. We will plot the estimate of the solution at each iteration. It is interesting to observe the iterates converge to the solution.

In [ ]:
u.interpolate(dolfin.Constant((0., 0.)))
B.t = 2.5*load_max/nsteps
newton_solver(u, monitor=plot_monitor)

### Illustrating quadratic convergence

We now want to verify that the Newton–Raphson iterations converge quadratically, that is $\epsilon_{n+1} \leq C\epsilon_n^2$, where $\epsilon_n$ denotes the error of the $n$-th iterate. Since we do not know the exact solution of the problem, we will take the last iterate as a reference.

We implement a new `monitor` that keeps a copy of each iterate.

In [ ]:
iterates = []
def my_monitor(iteration, u, u_norm, delta_u_norm, residual_norm):
    simple_monitor(iteration, u, u_norm, delta_u_norm, residual_norm)
    iterates.append(u.copy(deepcopy=True))

u.interpolate(dolfin.Constant((0., 0.)))
B.t = 2.5*load_max/nsteps
newton_solver(u, monitor=my_monitor, rtol=1e-10, atol=1e-10)

In [ ]:
L2_errors = np.zeros(len(iterates), dtype=np.float64)
H1_errors = np.zeros_like(L2_errors)
u_ref = iterates[-1]
for i, u_i in enumerate(iterates):
    L2_errors[i] = dolfin.errornorm(u_ref, u_i, 'L2')
    H1_errors[i] = dolfin.errornorm(u_ref, u_i, 'H1')

In [ ]:
plt.xlabel(r'$\epsilon_n^2$')
plt.ylabel(r'$\epsilon_{n+1}$')

plt.loglog(L2_errors[:-2]**2, L2_errors[1:-1], 'o-')
plt.loglog(L2_errors[:-2]**2, 100*L2_errors[:-2]**2)

In [ ]:
plt.xlabel(r'$\epsilon_n^2$')
plt.ylabel(r'$\epsilon_{n+1}$')

plt.loglog(H1_errors[:-2]**2, H1_errors[1:-1], 'o-')
plt.loglog(H1_errors[:-2]**2, 10*H1_errors[:-2]**2)

## Validation of the solver

We now run the whole simulation and compare with the reference solution.

In [ ]:
displ = np.zeros_like(loads)
u.interpolate(dolfin.Constant((0.,0.)))
file_u = dolfin.XDMFFile(os.path.join(output_dir, "u.xdmf"))
file_u.parameters.update({"flush_output":True,
                          "functions_share_mesh":True,
                          "rewrite_function_mesh":False})
for (i, t) in enumerate(loads):
    B.t = -t
    newton_solver(u)
    displ[i] = dolfin.assemble(u[1]*ds(2))/Ly 
    #print("t={:+3.3f}, u={:+3.3f}".format(t,displ_ref[i]))
    with file_u as file:
        file.write(u,t)

plt.plot(loads, displ_ref, "-", label='Reference implementation')
plt.plot(loads, displ, 'o', label='Our implementation')
plt.xlabel("load")
plt.ylabel("end-displacement")
plt.legend()